In [105]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.model_zoo as model_zoo
from torch.utils.data import DataLoader, Dataset
from torchvision import get_image_backend
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR

import numpy as np

from torchvision import models, transforms, datasets
from PIL import Image
import os

from os import listdir
from os.path import isfile, join
import pdb

In [114]:
def save_img(tensor_orig, tensor_transformed, style, filename):
    
    def recover(t):
        t = t.cpu().numpy()[0].transpose(1, 2, 0) * 255
        t = t.clip(0, 255).astype(np.uint8)
        return t
    
    result = Image.fromarray(recover(tensor_transformed))
    orig = Image.fromarray(recover(tensor_orig))
    style = Image.fromarray(recover(style))
    
    new_im = Image.new('RGB', (result.size[0] * 3 + 5 * 2, result.size[1]))
    new_im.paste(orig, (0, 0))
    new_im.paste(result, (result.size[0] + 5, 0))
    new_im.paste(style, (result.size[0] * 2 + 10, 0))
    new_im.save("/scratch/vr1059/style_transfer_debug_training/" + filename)

In [89]:
def recover_from_ImageNet(img):
    '''
    recover from ImageNet normalized rep to real img rep [0, 1]
    '''
    img *= torch.Tensor([0.229, 0.224, 0.225]
                        ).view(1, 3, 1, 1).expand_as(img)
    img += torch.Tensor([0.485, 0.456, 0.406]
                        ).view(1, 3, 1, 1).expand_as(img)

    return img

In [69]:
class PerceptualLoss(nn.Module):
    '''
    
    Implement perceptual (style) loss in a VGG network.
    
    ref:
    https://github.com/ceshine/fast-neural-style/blob/master/style-transfer.ipynb
    
    input: BxCxHxW, BxCxHxW
    output: loss type Variable
    
    '''
    def __init__(self, vgg_model):
        super(PerceptualLoss, self).__init__()
        self.vgg_layers = vgg_model.features
        
        # use relu_1_1, 2_1, 3_1, 4_1
        self.use_layer = set(['2', '9', '16', '29'])
        
    def forward(self, g, s):  
        loss = 0
        
        for name, module in self.vgg_layers._modules.items():
            g, s = module(g), module(s)
            if name in self.use_layer:
                g_mean, g_var = get_mean_var(g)
                s_mean, s_var = get_mean_var(s)
                s_mean = Variable(s_mean.data, requires_grad=False)
                s_var = Variable(s_var.data, requires_grad=False)
                loss += F.mse_loss(g_mean, s_mean, reduction='sum') + \
                        F.mse_loss(g_var, s_var, reduction='sum')
        return loss

In [3]:
def get_mean_var(t):
    # t for tensor
    
    n_batch, n_ch, h, w = t.size()

    t_view = t.view(n_batch, n_ch, h*w)
    t_mean = t_view.mean(dim=2)
    t_var = t_view.var(dim=2)

    # broadcast
    t_mean = t_mean.view(n_batch, n_ch, 1, 1).expand_as(t)
    t_var = t_var.view(n_batch, n_ch, 1, 1).expand_as(t)
    
    return t_mean, t_var

In [4]:
class AdaInstanceNormalization(nn.Module):
    
    '''
    
    style transfer in 'feature space'. 
    Combining the content image's feature maps and style image's feature maps together, channel-wise. 
        - weighting the style feature maps much more strongly. 
        - i wonder if this could be done by just adding feature maps together, or linearly combining them in some way
    
    '''
    
    def __init__(self, eps=1e-6):
        super(AdaInstanceNormalization, self).__init__()
        self.eps = eps
        
    def forward(self, c, s):
        '''
        c - content image's feature map
        s - style image's feature map
        '''
        c_mean, c_var = get_mean_var(c)
        s_mean, s_var = get_mean_var(s)
        
        return s_var * ((c - c_mean) / (c_var + self.eps)) + s_mean
        

In [5]:
class EncoderLayer(nn.Module):
    '''
    - construct partial VGG-19 model (through relu_4_1)
    - will fill in weights in separate method
    '''
    
    def __init__(self, batch_norm):
        super(EncoderLayer, self).__init__()
        
        # constructing partial architecture of VGG-19 (through relu_4_1)
        conf = models.vgg.cfg['E'][:12] 
        self.features = models.vgg.make_layers(conf, batch_norm=batch_norm)
        
    def forward(self, x):
        return self.features(x)
        

In [6]:
# When you initialize an nn.Module, it creates a state_dict automatically. 

In [7]:
def make_encoder(model_file, batch_norm=True):
    '''
    
    initialize partial VGG-19 model and fill with pre-trained weights
    
    '''
    VGG_TYPE = 'vgg19_bn' if batch_norm else 'vgg19'
    enc = EncoderLayer(batch_norm)
    
    if model_file and os.path.isfile(model_file):
        enc.load_state_dict(torch.load(model_file))
    else:
        vgg_weights = model_zoo.load_url(models.vgg.model_urls[VGG_TYPE])
        w = {}
        vgg_keys = vgg_weights.keys()
        for key in enc.state_dict().keys():
            if key in vgg_keys:
                w[key] = vgg_weights[key]
        enc.load_state_dict(w)
        if not model_file:
            model_file = "encoder.model"
        torch.save(enc.state_dict(), model_file)
        
    return enc

In [8]:
class DecoderLayer(nn.Module):
    '''
    
    A decoder that converts a set of feature maps to an image.  
    AdaIN layer --> decoder --> stylized image
    
    The reverse of VGG-19, with max-pooling replaced by upsampling. 
    
    '''
    def __init__(self):
        super(DecoderLayer, self).__init__()
        
        conf = [
            (1, 256),
            'U',
            (3, 256),
            'U',
            (1, 128),
            (1, 64),
            'U',
            (1, 64),
            (1, 3)
        ]
        
        self.features = self._make_layers(conf)
        
    def _make_layers(self, conf):
        layers = []
        in_channels = 512
        for block in conf:
            if block == 'U':
                layers += [nn.Upsample(scale_factor=2, mode='bilinear')]
                continue
            
            n_layer, n_feat = block
            for i in range(0, n_layer):
                layers += [
                    nn.ReflectionPad2d(1),
                    nn.Conv2d(in_channels, n_feat, kernel_size=3, stride=1),
                    nn.ReLU()
                ]
                in_channels = n_feat
        layers.pop()
        
        return nn.Sequential(*layers)
        
        
    def forward(self, x):
        return self.features(x)
        

In [9]:
def make_decoder(model_file):
    '''
    
    make a pre-trained partial VGG-19 network
    
    '''
    
    dec = DecoderLayer()
    if model_file and os.path.isfile(model_file):
        dec.load_state_dict(torch.load(model_file))
    else:
        raise ValueError('Decoder model not found.')
        
    return dec

In [10]:
class Args():
    epochs = 100
    batch_size = 8
    image_size = 256
    content_folder = '/scratch/vr1059/coco'
    style_folder = '/scratch/vr1059/painter_by_numbers'
    model_encoder = ''
    lr = 1e-4
    
args = Args()

In [11]:
# Transforms

image_transform = transforms.Compose([
    transforms.Resize(args.image_size),
    transforms.RandomCrop(args.image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
    # VGG was trained on images normalized with those statistics
])

style_transform = transforms.Compose([
    transforms.Resize(args.image_size),
    transforms.RandomCrop(args.image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
    # VGG was trained on images normalized with those statistics
])

image_transform_nocrop = transforms.Compose([
    transforms.Resize(args.image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

In [12]:
def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('RGB')

In [13]:
class ImageDataset(Dataset):
    
    def __init__(self, path, transform):
        """
        Args:
            path (string): Path to image directory. 
            transform (callable, optional): (optional) transform to be applied on a sample.
        """
        self.path = path
        self.fnames = [f for f in listdir(path)]
        self.transform = transform
        
    def __len__(self):
        return len(self.fnames)
        
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        img_name = self.fnames[idx]
        image = pil_loader(self.path + img_name)
        sample = self.transform(image)
        
        return sample

In [14]:
# content and style loader
content_train_dataset = ImageDataset(args.content_folder + '/train/', image_transform)
content_train_loader = DataLoader(
    content_train_dataset, batch_size=args.batch_size, shuffle=True)
content_validation_dataset = ImageDataset(args.content_folder + '/val/', image_transform)
content_validation_loader = DataLoader(
    content_validation_dataset, batch_size=args.batch_size)

In [15]:
style_train_dataset = ImageDataset(args.style_folder + '/train/', image_transform)
style_train_loader = DataLoader(
    style_train_dataset, batch_size=args.batch_size, shuffle=True)
style_validation_dataset = ImageDataset(args.style_folder + '/test/', image_transform)
style_validation_loader = DataLoader(
    style_validation_dataset, batch_size=args.batch_size)

In [16]:
enc = make_encoder('encoder.model')

In [17]:
adaIN = AdaInstanceNormalization()

In [18]:
dec = DecoderLayer()

In [19]:
perceptual_loss = PerceptualLoss(enc)

In [20]:
enc.features._modules.items()

odict_items([('0', Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))), ('1', BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)), ('2', ReLU(inplace)), ('3', Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))), ('4', BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)), ('5', ReLU(inplace)), ('6', MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)), ('7', Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))), ('8', BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)), ('9', ReLU(inplace)), ('10', Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))), ('11', BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)), ('12', ReLU(inplace)), ('13', MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)), ('14', Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))), 

In [21]:
for param in dec.parameters():
    if param.dim() < 2:
        continue
    nn.init.kaiming_normal_(param.data)

In [22]:
TORCH_SEED = 1080

In [23]:
torch.manual_seed(TORCH_SEED)

In [24]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device('cpu')

In [25]:
enc.to(device)
adaIN.to(device)
dec.to(device)
perceptual_loss.to(device)
torch.cuda.manual_seed(TORCH_SEED)
# torch.set_default_tensor_type('torch.cuda.FloatTensor')
optimizer = Adam(dec.parameters(), args.lr)

In [26]:
dec.train()
enc.eval()

EncoderLayer(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256, kernel_si

In [27]:
scheduler = StepLR(optimizer, step_size=1000, gamma=0.99)

In [91]:
SAVE_DEBUG_IMAGE_PER_NBATCH = 20

In [112]:
def train(epoch):
    enc.train()
    
    loader = zip(content_train_loader, style_train_loader)
    
    avg_closs = avg_sloss = avg_loss = 0
    
    n_samples = 0
    
    for (i, (xx, ss)) in enumerate(loader):
        
        x = Variable(xx).to(device)
        s = Variable(ss).to(device)
        
        optimizer.zero_grad()
        
        c_feat, s_feat = enc(x), enc(s)
        
        t = adaIN(c_feat, s_feat)
        
        gen_img = dec(t)
        
        ft = enc(gen_img)
        
        content_loss = F.mse_loss(ft, t, reduction='sum')
        style_loss = perceptual_loss(gen_img, s.expand_as(gen_img))
        loss = content_loss + 0.1 * style_loss
                
        batch_closs, batch_sloss, batch_loss = content_loss, style_loss, loss 
        
        n_samples += len(xx)
        
        loss.backward()
        optimizer.step()
        
        if i % SAVE_DEBUG_IMAGE_PER_NBATCH == 0:
            print("Train Epoch {}: Batch {} content: {} style: {} loss: {}".format(epoch, i, batch_closs/len(xx), batch_sloss/len(xx), batch_loss/len(xx)))
            
            save_img(recover_from_ImageNet(x.data.cpu()), recover_from_ImageNet(gen_img.data.cpu()), recover_from_ImageNet(s.data.cpu()), "debug_train_{}_{}.png".format(epoch, i))        

In [ ]:
for epoch in range(args.epochs):
    train(epoch)

/home/vr1059/py3.6.3/lib/python3.6/site-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.Upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))
/home/vr1059/py3.6.3/lib/python3.6/site-packages/torch/nn/functional.py:2423: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/home/vr1059/py3.6.3/lib/python3.6/site-packages/torch/nn/_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Train Epoch 0: Batch 0 content: 14916.53515625 style: 9951.9365234375 loss: 15911.728515625
Train Epoch 0: Batch 20 content: 10981.8134765625 style: 9752.4482421875 loss: 11957.05859375
Train Epoch 0: Batch 40 content: 11706.8330078125 style: 10138.380859375 loss: 12720.6708984375
Train Epoch 0: Batch 60 content: 15678.97265625 style: 10924.1494140625 loss: 16771.38671875
Train Epoch 0: Batch 80 content: 10249.3837890625 style: 10984.7080078125 loss: 11347.8544921875
Train Epoch 0: Batch 100 content: 12369.9375 style: 11034.802734375 loss: 13473.41796875
Train Epoch 0: Batch 120 content: 14503.009765625 style: 7073.50048828125 loss: 15210.359375
Train Epoch 0: Batch 140 content: 12132.37890625 style: 7628.33984375 loss: 12895.212890625
Train Epoch 0: Batch 160 content: 11995.4453125 style: 8303.0244140625 loss: 12825.748046875
Train Epoch 0: Batch 180 content: 9888.6103515625 style: 10764.60546875 loss: 10965.0712890625
Train Epoch 0: Batch 200 content: 12642.94140625 style: 5868.79931

Train Epoch 0: Batch 1800 content: 13786.5634765625 style: 3960.9658203125 loss: 14182.66015625
Train Epoch 0: Batch 1820 content: 9505.1337890625 style: 3376.942626953125 loss: 9842.828125
Train Epoch 0: Batch 1840 content: 7570.62158203125 style: 3339.636474609375 loss: 7904.58544921875
Train Epoch 0: Batch 1860 content: 14534.1669921875 style: 4977.818359375 loss: 15031.94921875
Train Epoch 0: Batch 1880 content: 7262.0478515625 style: 3040.476318359375 loss: 7566.095703125
Train Epoch 0: Batch 1900 content: 8573.0673828125 style: 5945.01513671875 loss: 9167.5693359375
Train Epoch 0: Batch 1920 content: 18840.49609375 style: 4653.5244140625 loss: 19305.84765625
Train Epoch 0: Batch 1940 content: 9218.375 style: 4289.42822265625 loss: 9647.3173828125
Train Epoch 0: Batch 1960 content: 7508.49658203125 style: 3853.922119140625 loss: 7893.888671875
Train Epoch 0: Batch 1980 content: 26678.8046875 style: 5123.2724609375 loss: 27191.1328125
Train Epoch 0: Batch 2000 content: 8509.9541015

Train Epoch 0: Batch 3620 content: 5995.4716796875 style: 2764.83349609375 loss: 6271.955078125
Train Epoch 0: Batch 3640 content: 5603.1044921875 style: 2655.77978515625 loss: 5868.6826171875
Train Epoch 0: Batch 3660 content: 16665.7421875 style: 4061.98291015625 loss: 17071.94140625
Train Epoch 0: Batch 3680 content: 6597.08056640625 style: 2648.440673828125 loss: 6861.9248046875
Train Epoch 0: Batch 3700 content: 6716.962890625 style: 2760.108154296875 loss: 6992.9736328125
Train Epoch 0: Batch 3720 content: 6024.44384765625 style: 3746.97265625 loss: 6399.14111328125
Train Epoch 0: Batch 3740 content: 7473.87060546875 style: 3447.341552734375 loss: 7818.60498046875
Train Epoch 0: Batch 3760 content: 20901.7265625 style: 3143.1181640625 loss: 21216.0390625
Train Epoch 0: Batch 3780 content: 5857.458984375 style: 2985.51513671875 loss: 6156.0107421875
Train Epoch 0: Batch 3800 content: 10349.923828125 style: 3917.008544921875 loss: 10741.625
Train Epoch 0: Batch 3820 content: 9064.3

Train Epoch 0: Batch 5340 content: 12214.7470703125 style: 3819.072509765625 loss: 12596.654296875
Train Epoch 0: Batch 5360 content: 11811.7099609375 style: 2684.432373046875 loss: 12080.1533203125
Train Epoch 0: Batch 5380 content: 6966.396484375 style: 2668.526123046875 loss: 7233.2490234375
Train Epoch 0: Batch 5400 content: 230766.859375 style: 10274.0341796875 loss: 231794.265625
Train Epoch 0: Batch 5420 content: 8828.0546875 style: 2505.240966796875 loss: 9078.5791015625
Train Epoch 0: Batch 5440 content: 5934.45361328125 style: 3320.63623046875 loss: 6266.51708984375
Train Epoch 0: Batch 5460 content: 32327.08984375 style: 3721.9609375 loss: 32699.28515625
Train Epoch 0: Batch 5480 content: 6954.75244140625 style: 2531.1591796875 loss: 7207.8681640625
Train Epoch 0: Batch 5500 content: 7568.97900390625 style: 2971.529052734375 loss: 7866.1318359375
Train Epoch 0: Batch 5520 content: 8985.888671875 style: 3506.98388671875 loss: 9336.5869140625
Train Epoch 0: Batch 5540 content:

...

Train Epoch 1: Batch 560 content: 6120.580078125 style: 2444.92236328125 loss: 6365.072265625
Train Epoch 1: Batch 580 content: 7778.853515625 style: 2712.29931640625 loss: 8050.08349609375
Train Epoch 1: Batch 600 content: 7378.880859375 style: 2379.00048828125 loss: 7616.78076171875
Train Epoch 1: Batch 620 content: 8174.96875 style: 2737.95166015625 loss: 8448.763671875
Train Epoch 1: Batch 640 content: 6088.14453125 style: 1930.6002197265625 loss: 6281.20458984375
Train Epoch 1: Batch 660 content: 10987.361328125 style: 3128.235595703125 loss: 11300.1845703125
Train Epoch 1: Batch 680 content: 14094.8671875 style: 3345.682373046875 loss: 14429.435546875
Train Epoch 1: Batch 700 content: 24180.15234375 style: 2289.49853515625 loss: 24409.1015625
Train Epoch 1: Batch 720 content: 6611.97314453125 style: 2560.990478515625 loss: 6868.072265625
Train Epoch 1: Batch 740 content: 6467.884765625 style: 2749.253173828125 loss: 6742.81005859375
Train Epoch 1: Batch 760 content: 9399.48046875

Train Epoch 1: Batch 2280 content: 4971.3154296875 style: 1798.1226806640625 loss: 5151.1279296875
Train Epoch 1: Batch 2300 content: 5749.6328125 style: 2371.055908203125 loss: 5986.73828125
Train Epoch 1: Batch 2320 content: 7536.30859375 style: 2278.746826171875 loss: 7764.18310546875
Train Epoch 1: Batch 2340 content: 8560.6513671875 style: 2511.339599609375 loss: 8811.78515625
Train Epoch 1: Batch 2360 content: 11612.8193359375 style: 1948.989990234375 loss: 11807.71875
Train Epoch 1: Batch 2380 content: 5244.31396484375 style: 2321.078857421875 loss: 5476.421875
Train Epoch 1: Batch 2400 content: 4896.0126953125 style: 2413.448974609375 loss: 5137.357421875
Train Epoch 1: Batch 2420 content: 5456.650390625 style: 2247.4794921875 loss: 5681.3984375
Train Epoch 1: Batch 2440 content: 8152.2119140625 style: 2621.757080078125 loss: 8414.3876953125
Train Epoch 1: Batch 2460 content: 6356.708984375 style: 1830.6964111328125 loss: 6539.77880859375
Train Epoch 1: Batch 2480 content: 7850

Train Epoch 1: Batch 4000 content: 5676.89599609375 style: 3565.2109375 loss: 6033.4169921875
Train Epoch 1: Batch 4020 content: 5386.900390625 style: 2251.285888671875 loss: 5612.02880859375
Train Epoch 1: Batch 4040 content: 20402.83203125 style: 2547.7822265625 loss: 20657.609375
Train Epoch 1: Batch 4060 content: 5037.822265625 style: 5362.30322265625 loss: 5574.052734375
Train Epoch 1: Batch 4080 content: 5915.9375 style: 2152.7822265625 loss: 6131.2158203125
Train Epoch 1: Batch 4100 content: 8277.9970703125 style: 2147.271728515625 loss: 8492.724609375
Train Epoch 1: Batch 4120 content: 6269.2919921875 style: 2351.087158203125 loss: 6504.40087890625
Train Epoch 1: Batch 4140 content: 7837.3818359375 style: 1980.1854248046875 loss: 8035.400390625
Train Epoch 1: Batch 4160 content: 8800.5078125 style: 2841.50439453125 loss: 9084.658203125
Train Epoch 1: Batch 4180 content: 4543.396484375 style: 1799.5177001953125 loss: 4723.34814453125
Train Epoch 1: Batch 4200 content: 5686.54687

Train Epoch 1: Batch 5720 content: 7002.3056640625 style: 2739.446044921875 loss: 7276.25048828125
Train Epoch 1: Batch 5740 content: 4680.8486328125 style: 1832.9765625 loss: 4864.146484375
Train Epoch 1: Batch 5760 content: 10028.423828125 style: 1992.2952880859375 loss: 10227.6533203125
Train Epoch 1: Batch 5780 content: 5039.7431640625 style: 2526.271484375 loss: 5292.3701171875
Train Epoch 1: Batch 5800 content: 8121.357421875 style: 2419.152099609375 loss: 8363.2724609375
Train Epoch 1: Batch 5820 content: 31894.5 style: 2954.033447265625 loss: 32189.904296875
Train Epoch 1: Batch 5840 content: 16366.2060546875 style: 2789.759033203125 loss: 16645.181640625
Train Epoch 1: Batch 5860 content: 5346.71337890625 style: 3238.38037109375 loss: 5670.55126953125
Train Epoch 1: Batch 5880 content: 11314.7412109375 style: 6155.9375 loss: 11930.3349609375
Train Epoch 1: Batch 5900 content: 5456.3125 style: 2243.59423828125 loss: 5680.671875
Train Epoch 1: Batch 5920 content: 5172.642578125 

Train Epoch 1: Batch 7440 content: 10017.373046875 style: 2804.03759765625 loss: 10297.7763671875
Train Epoch 1: Batch 7460 content: 5328.8779296875 style: 2328.1083984375 loss: 5561.68896484375
Train Epoch 1: Batch 7480 content: 6437.89892578125 style: 2995.88037109375 loss: 6737.48681640625
Train Epoch 1: Batch 7500 content: 13195.8583984375 style: 1985.6142578125 loss: 13394.419921875
Train Epoch 1: Batch 7520 content: 6193.498046875 style: 3690.99755859375 loss: 6562.59765625
Train Epoch 1: Batch 7540 content: 9579.4716796875 style: 2462.29150390625 loss: 9825.701171875
Train Epoch 1: Batch 7560 content: 8632.3994140625 style: 2506.86865234375 loss: 8883.0859375
Train Epoch 1: Batch 7580 content: 4261.6015625 style: 1994.6612548828125 loss: 4461.06787109375
Train Epoch 1: Batch 7600 content: 8629.189453125 style: 2100.8486328125 loss: 8839.2744140625
Train Epoch 1: Batch 7620 content: 5734.00146484375 style: 1690.09228515625 loss: 5903.0107421875
Train Epoch 1: Batch 7640 content: 